## Part A : Q1 -Q3

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import pytorch_lightning as pl
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, random_split
import wandb
from pytorch_lightning.loggers import WandbLogger
from pytorch_lightning.callbacks import ModelCheckpoint
from kaggle_secrets import UserSecretsClient # used in kaggle to support wandb 
from data_loading import INaturalistDataModule
from simpleCNN import SimpleCNN

In [ ]:
################################################################
# Preparing Dataset with and without augmentation 
################################################################
class INaturalistDataModule(pl.LightningDataModule):
    def __init__(self, data_dir, batch_size=32, num_workers=4, valid_split=0.2, use_augmentation=True):
        super().__init__()
        self.data_dir = data_dir
        self.batch_size = batch_size
        self.num_workers = num_workers
        self.valid_split = valid_split
        self.use_augmentation = use_augmentation  # Enable/Disable augmentation

        # Mean and std from ImageNet (often used for natural image datasets)
        self.mean = [0.485, 0.456, 0.406]
        self.std = [0.229, 0.224, 0.225]

        # **Train Transform with Data Augmentation**
        if self.use_augmentation:
            self.train_transform = transforms.Compose([
                transforms.Resize((224, 224)),  
                transforms.RandomHorizontalFlip(),  # Flip images randomly
                transforms.RandomRotation(10),  # Rotate within + or - 10 degrees
                transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),  # Adjust colors
                transforms.ToTensor(),
                transforms.Normalize(self.mean, self.std)  # Normalize
            ])
        else:
            self.train_transform = transforms.Compose([
                transforms.Resize((224, 224)),
                transforms.ToTensor(),
                transforms.Normalize(self.mean, self.std)
            ])

        # **Validation & Test Transform (No Augmentation)**
        self.test_transform = transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
            transforms.Normalize(self.mean, self.std)
        ])
    
    def setup(self, stage=None):
        # Load dataset
        full_train_dataset = torchvision.datasets.ImageFolder(root=f'{self.data_dir}/train', transform=self.train_transform)
        self.test_dataset = torchvision.datasets.ImageFolder(root=f'{self.data_dir}/val', transform=self.test_transform)

        # **Split train dataset into train (80%) & validation (20%)**
        total = len(full_train_dataset)
        val_size = int(total * self.valid_split)
        train_size = total - val_size
        self.train_dataset, self.val_dataset = random_split(full_train_dataset, [train_size, val_size])

        # **Set validation transform separately**
        self.val_dataset.dataset.transform = self.test_transform
        return self.train_dataset,self.val_dataset,self.test_dataset

    def train_dataloader(self):
        return DataLoader(self.train_dataset, batch_size=self.batch_size, shuffle=True, num_workers=self.num_workers)

    def val_dataloader(self):
        return DataLoader(self.val_dataset, batch_size=self.batch_size, shuffle=False, num_workers=self.num_workers)

    def test_dataloader(self):
        return DataLoader(self.test_dataset, batch_size=self.batch_size, shuffle=False, num_workers=self.num_workers)

Found 8001 images belonging to 10 classes.
Found 1999 images belonging to 10 classes.


In [ ]:
###############################################
# Listing the hyperparameters in wandb config 
###############################################
sweep_config = {
    'method': 'bayes',
    'metric': {
      'name': 'val_acc',
      'goal': 'maximize'   
    },
    'parameters': {
        'conv_filters': {
            'values': [[32, 32, 32, 32, 32], [32, 64, 128, 256, 512]]
        },
        'conv_kernel_sizes': {
            'values': [[3, 3, 3, 3, 3], [5, 3, 3, 3, 3]]
        },
        'conv_activation': {
            'values': ['ReLU', 'SiLU', 'GeLU']
        },
        'dense_neurons': {
            'values': [128, 256]
        },
        'dense_activation': {
            'values': ['ReLU', 'SiLU']
        },
        'dropout': {
            'values': [0.2, 0.3]
        },
        'use_batchnorm': {
            'values': [True, False]
        },
        'lr': {
            'values': [1e-3, 1e-4]
        },
        'batch_size': {
            'values': [32, 64]
        },
        'use_augmentation': {  
            'values': [True, False]
        }
    }
}

In [ ]:
#####################################
# Initializing the model architecture 
# Simple CNN Model with flexibile  conv blocks, each: Conv -> (optional BatchNorm) -> Activation -> MaxPool
#####################################
class SimpleCNN(pl.LightningModule):
    def __init__(self, conv_filters=[32, 32, 32, 32, 32],
                 conv_kernel_sizes=[3, 3, 3, 3, 3],
                 conv_activation='ReLU',
                 dense_neurons=128,
                 dense_activation='ReLU',
                 dropout=0.0,
                 use_batchnorm=False,
                 lr=1e-3):
        super(SimpleCNN, self).__init__()
        self.save_hyperparameters()
        # Dictionary mapping activation names to modules
        activations = {
            'ReLU': nn.ReLU,
            'GELU': nn.GELU,
            'SiLU': nn.SiLU
            
        }
        conv_act = activations.get(conv_activation, nn.ReLU)
        dense_act = activations.get(dense_activation, nn.ReLU)
        
        self.conv_layers = nn.ModuleList()
        in_channels = 3  # iNaturalist images have 3 channels (RGB)
        for i in range(5):
            out_channels = conv_filters[i]
            kernel_size = conv_kernel_sizes[i]
            conv = nn.Conv2d(in_channels, out_channels, kernel_size=kernel_size, padding=kernel_size//2)
            block = [conv]
            if use_batchnorm:
                block.append(nn.BatchNorm2d(out_channels))
            block.append(conv_act())
            block.append(nn.MaxPool2d(2))
            self.conv_layers.append(nn.Sequential(*block))
            in_channels = out_channels
        
        # Compute the flattened feature size after conv layers
        # Assume a default input image size of 224x224
        dummy_input = torch.zeros(1, 3, 224, 224)
        with torch.no_grad():
            features = dummy_input
            for layer in self.conv_layers:
                features = layer(features)
            self.flattened_size = features.view(1, -1).shape[1]
        
        # Dense layer followed by the output layer (10 neurons for 10 classes)
        self.dense = nn.Linear(self.flattened_size, dense_neurons)
        self.dense_activation = dense_act()
        self.dropout = nn.Dropout(dropout) if dropout > 0 else None
        self.out = nn.Linear(dense_neurons, 10)
        self.lr = lr

    def forward(self, x):
        for layer in self.conv_layers:
            x = layer(x)
        x = torch.flatten(x, 1)
        x = self.dense(x)
        x = self.dense_activation(x)
        if self.dropout:
            x = self.dropout(x)
        x = self.out(x)
        return x
    
    def training_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = F.cross_entropy(logits, y)
        acc = (logits.argmax(dim=1) == y).float().mean()
        self.log('train_loss', loss)
        self.log('train_acc', acc)
        return loss
    
    def validation_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = F.cross_entropy(logits, y)
        acc = (logits.argmax(dim=1) == y).float().mean()
        self.log('val_loss', loss, prog_bar=True)
        self.log('val_acc', acc, prog_bar=True)
    
    def test_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = F.cross_entropy(logits, y)
        acc = (logits.argmax(dim=1) == y).float().mean()
        self.log('test_loss', loss)
        self.log('test_acc', acc)
    
    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=self.lr)
        return optimizer

In [ ]:
#################################
# Setting up wandb sweeps
#################################
def train():
    # Initialize wandb run
    wandb.init(project="Simple_cnn")  
    config = wandb.config

    # print config to verify keys exist
    print("Loaded Config:", config)

    
    model = SimpleCNN(
         # Default filters
        conv_filters=config.get("conv_filters", [32, 32, 32, 32, 32]), 
        conv_kernel_sizes=config.get("conv_kernel_sizes", [3, 3, 3, 3, 3]),
        conv_activation=config.get("conv_activation", "ReLU"),
        dense_neurons=config.get("dense_neurons", 128),
        dense_activation=config.get("dense_activation", "ReLU"),
        dropout=config.get("dropout", 0.2),
        use_batchnorm=config.get("use_batchnorm", False),
        lr=config.get("lr", 1e-3)
    )

    data_module = INaturalistDataModule(
        data_dir='/kaggle/input/inaturalist-dataset/inaturalist_12K',
        batch_size=config.get("batch_size", 32),
        use_augmentation=config.get("use_augmentation", False) 
    )
    data_module.setup()
    train_loader = data_module.train_dataloader()
    val_loader = data_module.val_dataloader()
    test_loader = data_module.test_dataloader()

    # Log training process with wandb
    wandb_logger = WandbLogger(project="Simple_cnn", log_model="all")

    # Define Trainer pytorch_lighting method which automatically trains model and logs metrics 
    trainer = pl.Trainer(
        max_epochs=10,   
        logger=wandb_logger,
    )

    # Train the model
    trainer.fit(model, train_loader,val_loader)
 
if __name__ == "__main__":
    
    user_secrets = UserSecretsClient() #if using in  kaggle set up the wandb key and use that key to access wandb 
    secret_value_0 = user_secrets.get_secret("wandb")
    wandb.login(key=secret_value_0)
    sweep_id = wandb.sweep(sweep_config,project='Simple_cnn')
    
    wandb.agent(sweep_id, train, count=30)  # Runs 30 experiments

Create sweep with ID: firnynx9
Sweep URL: https://wandb.ai/cs6910-team/DL-Assignment2-PartA-5April-2/sweeps/firnynx9


wandb: Agent Starting Run: 75xanm09 with config:
wandb: 	activation: relu
wandb: 	batch_norm: True
wandb: 	conv_layers: 5
wandb: 	data_aug: True
wandb: 	dense_size: 64
wandb: 	dropout: 0.3
wandb: 	filter_org: 1
wandb: 	first_layer_filters: 32
wandb: 	kernel_size: 3
wandb: 	num_epochs: 50
wandb: 	optimizer: adam
wandb: Currently logged in as: cs6910-team (use `wandb login --relogin` to force relogin)


Epoch 1/50
62/62 [==============================] - 104s 1s/step - loss: 2.6069 - acc: 0.1224 - val_loss: 2.3555 - val_acc: 0.0958
Epoch 2/50
62/62 [==============================] - 71s 1s/step - loss: 2.3364 - acc: 0.1605 - val_loss: 2.4004 - val_acc: 0.1104
Epoch 3/50
62/62 [==============================] - 71s 1s/step - loss: 2.2678 - acc: 0.1966 - val_loss: 2.4575 - val_acc: 0.1250
Epoch 4/50
62/62 [==============================] - 72s 1s/step - loss: 2.2523 - acc: 0.1784 - val_loss: 2.3998 - val_acc: 0.0979
Epoch 5/50
62/62 [==============================] - 70s 1s/step - loss: 2.1841 - acc: 0.2064 - val_loss: 2.2999 - val_acc: 0.1210
Epoch 6/50
62/62 [==============================] - 71s 1s/step - loss: 2.2008 - acc: 0.2021 - val_loss: 2.3175 - val_acc: 0.1354
Epoch 7/50
62/62 [==============================] - 71s 1s/step - loss: 2.1811 - acc: 0.2222 - val_loss: 2.1949 - val_acc: 0.1896
Epoch 8/50
62/62 [==============================] - 70s 1s/step - loss: 2.1688 - acc: 0.2

wandb: Sweep Agent: Waiting for job.
wandb: Job received.
wandb: Agent Starting Run: 5dd724e3 with config:
wandb: 	activation: relu
wandb: 	batch_norm: True
wandb: 	conv_layers: 5
wandb: 	data_aug: True
wandb: 	dense_size: 64
wandb: 	dropout: 0
wandb: 	filter_org: 0.5
wandb: 	first_layer_filters: 64
wandb: 	kernel_size: 3
wandb: 	num_epochs: 50
wandb: 	optimizer: adam


Epoch 1/50
62/62 [==============================] - 73s 1s/step - loss: 2.4666 - acc: 0.1136 - val_loss: 2.3192 - val_acc: 0.1063
Epoch 2/50
62/62 [==============================] - 70s 1s/step - loss: 2.2919 - acc: 0.1391 - val_loss: 2.4047 - val_acc: 0.0667
Epoch 3/50
62/62 [==============================] - 71s 1s/step - loss: 2.2272 - acc: 0.1767 - val_loss: 2.3916 - val_acc: 0.1058
Epoch 4/50
62/62 [==============================] - 74s 1s/step - loss: 2.1970 - acc: 0.1920 - val_loss: 2.4862 - val_acc: 0.0750
Epoch 5/50
62/62 [==============================] - 74s 1s/step - loss: 2.2016 - acc: 0.1876 - val_loss: 2.5452 - val_acc: 0.0771
Epoch 6/50
62/62 [==============================] - 73s 1s/step - loss: 2.1760 - acc: 0.1954 - val_loss: 2.4215 - val_acc: 0.1250
Epoch 7/50
62/62 [==============================] - 71s 1s/step - loss: 2.1532 - acc: 0.2036 - val_loss: 2.3196 - val_acc: 0.1404
Epoch 8/50
62/62 [==============================] - 73s 1s/step - loss: 2.1276 - acc: 0.21

...

wandb: Agent Starting Run: 3vvzrj50 with config:
wandb: 	activation: relu
wandb: 	batch_norm: True
wandb: 	conv_layers: 5
wandb: 	data_aug: True
wandb: 	dense_size: 128
wandb: 	dropout: 0.3
wandb: 	filter_org: 0.5
wandb: 	first_layer_filters: 64
wandb: 	kernel_size: 3
wandb: 	num_epochs: 50
wandb: 	optimizer: adam


Epoch 1/50
62/62 [==============================] - 74s 1s/step - loss: 2.7035 - acc: 0.1110 - val_loss: 2.3119 - val_acc: 0.1000
Epoch 2/50
62/62 [==============================] - 72s 1s/step - loss: 2.4500 - acc: 0.1290 - val_loss: 2.3546 - val_acc: 0.0771
Epoch 3/50
62/62 [==============================] - 72s 1s/step - loss: 2.3408 - acc: 0.1500 - val_loss: 2.3975 - val_acc: 0.1274
Epoch 4/50
62/62 [==============================] - 74s 1s/step - loss: 2.3413 - acc: 0.1634 - val_loss: 2.3960 - val_acc: 0.1042
Epoch 5/50
62/62 [==============================] - 73s 1s/step - loss: 2.2675 - acc: 0.1921 - val_loss: 2.3840 - val_acc: 0.0958
Epoch 6/50
62/62 [==============================] - 72s 1s/step - loss: 2.2526 - acc: 0.1805 - val_loss: 2.4192 - val_acc: 0.1167
Epoch 7/50
62/62 [==============================] - 72s 1s/step - loss: 2.2256 - acc: 0.1954 - val_loss: 2.3030 - val_acc: 0.1490
Epoch 8/50
62/62 [==============================] - 73s 1s/step - loss: 2.2145 - acc: 0.17

wandb: Agent Starting Run: 3sd2cunk with config:
wandb: 	activation: relu
wandb: 	batch_norm: True
wandb: 	conv_layers: 5
wandb: 	data_aug: True
wandb: 	dense_size: 32
wandb: 	dropout: 0.2
wandb: 	filter_org: 1
wandb: 	first_layer_filters: 32
wandb: 	kernel_size: 3
wandb: 	num_epochs: 50
wandb: 	optimizer: adam


Epoch 1/50
     22/Unknown - 24s 1s/step - loss: 2.6613 - acc: 0.1099

wandb: Ctrl + C detected. Stopping sweep.
